In [1]:
import torch
import torchvision

import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from pytorch_msssim import ssim
import torchvision.transforms as T
from PIL import Image
import datetime
import time
import os

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.cuda.memory_allocated())

True
NVIDIA RTX A6000
0


# Choose variants here:

In [3]:
trained_model = False
arch = 'ResNet18'

## System setup:

In [4]:
import inversefed
setup = inversefed.utils.system_startup()
defs = inversefed.training_strategy('conservative')

loss_fn, trainloader, validloader =  inversefed.construct_dataloaders('ImageNet', defs, 
                                                                      data_path=r'inversefed/data/Brain-Tumor-MRI-Dataset')

model = torchvision.models.resnet18(pretrained=trained_model)
model.to(**setup)
model.eval();

Currently evaluating -------------------------------:
Tuesday, 29. August 2023 11:52AM
CPUs: 32, GPUs: 2 on solidlab-server.
GPU : NVIDIA RTX A6000
Dataset ImageNet
    Number of datapoints: 5712
    Root location: inversefed/data/Brain-Tumor-MRI-Dataset
    Split: train
    StandardTransform
Transform: ToTensor()


/lclhome/bdas004/anaconda3/envs/GPU_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/lclhome/bdas004/anaconda3/envs/GPU_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
dm = torch.as_tensor(inversefed.consts.cifar10_mean, **setup)[:, None, None]
ds = torch.as_tensor(inversefed.consts.cifar10_std, **setup)[:, None, None]
def plot(tensor):
    tensor = tensor.clone().detach()
    tensor.mul_(ds).add_(dm).clamp_(0, 1)
    if tensor.shape[0] == 1:
        return plt.imshow(tensor[0].permute(1, 2, 0).cpu());
    else:
        fig, axes = plt.subplots(1, tensor.shape[0], figsize=(12, tensor.shape[0]*12))
        for i, im in enumerate(tensor):
            axes[i].imshow(im.permute(1, 2, 0).cpu());

In [6]:
len(trainloader.dataset)

5712

# Reconstruct

In [7]:
output_folder = "Brain_MRI_input_and_recovered_images_for_Resnet_untrained"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

### Build the input (ground-truth) gradient

# For 100 Images

In [ ]:
test_mse_final=0
sSim_final=0
whole_duration=0
loss_value=0
count=0
ASR=0
for i in range (57, len(trainloader.dataset),57):
    print("Image No.", count+1)
    print("For image : ", i)
    
    idx = i 
    # 8112 # the beagle
    # 1200 # the owl
    # 11794 # the German shepherd
    # 19449 # the panda


    # np.random.randint(len(validloader.dataset))

    img, label = trainloader.dataset[idx]
    labels = torch.as_tensor((label,), device=setup['device'])
    ground_truth = img.to(**setup).unsqueeze(0)
   
    #plot(ground_truth);
    print([trainloader.dataset.classes[l] for l in labels]);


    ground_truth_denormalized = torch.clamp(ground_truth * ds + dm, 0, 1)
    output_path = os.path.join(output_folder, f'{idx}_{arch}_Brain_MRI_untrained_input_loop_test.png')
    torchvision.utils.save_image(ground_truth_denormalized, output_path)
    
    
    
    #torchvision.utils.save_image(ground_truth_denormalized, f'{idx}_{arch}_covid_19_untrained_input_loop_test.png')

    
    model.zero_grad()
    target_loss, _, _ = loss_fn(model(ground_truth), labels)
    input_gradient = torch.autograd.grad(target_loss, model.parameters())
    input_gradient = [grad.detach() for grad in input_gradient]
    full_norm = torch.stack([g.norm() for g in input_gradient]).mean()
    print(f'Full gradient norm is {full_norm:e}.')
    config = dict(signed=True,
                  boxed=True,
                  cost_fn='sim',
                  indices='top10',
                  weights='equal',
                  lr=0.1,
                  optim='adam',
                  restarts=8,
                  max_iterations=24000,
                  total_variation=1e-6,
                  init='randn',
                  filter='median',
                  lr_decay=True,
                  scoring_choice='loss')
    start_time = time.time()
    rec_machine = inversefed.GradientReconstructor(model, (dm, ds), config, num_images=1)
    output, stats = rec_machine.reconstruct(input_gradient, labels, img_shape=(3, 224, 224))
    
    test_mse = (output.detach() - ground_truth).pow(2).mean()
    test_mse_final=test_mse_final+test_mse
    feat_mse = (model(output.detach())- model(ground_truth)).pow(2).mean()  
    test_psnr = inversefed.metrics.psnr(output, ground_truth, factor=1/ds)
    sSim=ssim(output, ground_truth).item()
    sSim_final=sSim_final+sSim
    loss_value=loss_value+stats['opt']
    plot(output)
    
    plt.title(f"Rec. loss: {stats['opt']:2.4f} | MSE: {test_mse:2.4f} | SSIM: {sSim:2.4f}"
              f"| PSNR: {test_psnr:4.2f} | FMSE: {feat_mse:2.4e} |");
    
    output_denormalized = torch.clamp(output * ds + dm, 0, 1)
    output_path = os.path.join(output_folder, f'{idx}_{arch}_Brain_MRI_untrained_output_loop_test.png')
    torchvision.utils.save_image(output_denormalized, output_path)
    #torchvision.utils.save_image(output_denormalized, f'{idx}_{arch}_covid_19_untrained_output_loop_test.png')
    
    
    

    
    data = inversefed.metrics.activation_errors(model, output, ground_truth)
    fig, axes = plt.subplots(2, 4, sharey=False, figsize=(14,8))
    axes[0, 0].semilogy(list(data['se'].values())[:-3])
    axes[0, 0].set_title('SE')
    axes[0, 1].semilogy(list(data['mse'].values())[:-3])
    axes[0, 1].set_title('MSE')
    axes[0, 2].plot(list(data['sim'].values())[:-3])
    axes[0, 2].set_title('Similarity')

    convs = [val for key, val in data['mse'].items() if 'conv' in key]
    axes[1, 0].semilogy(convs)
    axes[1, 0].set_title('MSE - conv layers')
    convs = [val for key, val in data['mse'].items() if 'conv1' in key]
    axes[1, 1].semilogy(convs)
    convs = [val for key, val in data['mse'].items() if 'conv2' in key]
    axes[1, 1].semilogy(convs)
    axes[1, 1].set_title('MSE - conv1 vs conv2 layers')
    bns = [val for key, val in data['mse'].items() if 'bn' in key]
    axes[1, 2].plot(bns)
    axes[1, 2].set_title('MSE - bn layers')
    fig.suptitle('Error between layers');
    
    
    end_time=time.time()-.1
    duration=end_time-start_time
    whole_duration=whole_duration+duration
    
    print("Time to recover Image", i, ": ",duration)
    if(sSim>=0.9):
        ASR=ASR+1
    print()
    count=count+1
    
    

Image No. 1
For image :  57
[('tench', 'Tinca tinca')]


/lclhome/bdas004/anaconda3/envs/GPU_env/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Full gradient norm is 5.118896e+00.
It: 0. Rec. loss: 0.1396.
It: 500. Rec. loss: 0.0588.
It: 1000. Rec. loss: 0.0594.
It: 1500. Rec. loss: 0.0625.
It: 2000. Rec. loss: 0.0557.
It: 2500. Rec. loss: 0.0599.
It: 3000. Rec. loss: 0.0563.
It: 3500. Rec. loss: 0.0610.
It: 4000. Rec. loss: 0.0635.
It: 4500. Rec. loss: 0.0582.
It: 5000. Rec. loss: 0.0573.
It: 5500. Rec. loss: 0.0596.
It: 6000. Rec. loss: 0.0581.
It: 6500. Rec. loss: 0.0633.
It: 7000. Rec. loss: 0.0565.
It: 7500. Rec. loss: 0.0601.
It: 8000. Rec. loss: 0.0592.
It: 8500. Rec. loss: 0.0571.
It: 9000. Rec. loss: 0.0618.
It: 9500. Rec. loss: 0.0502.
It: 10000. Rec. loss: 0.0558.
It: 10500. Rec. loss: 0.0559.
It: 11000. Rec. loss: 0.0556.
It: 11500. Rec. loss: 0.0548.
It: 12000. Rec. loss: 0.0482.
It: 12500. Rec. loss: 0.0491.
It: 13000. Rec. loss: 0.0489.
It: 13500. Rec. loss: 0.0511.
It: 14000. Rec. loss: 0.0531.
It: 14500. Rec. loss: 0.0512.
It: 15000. Rec. loss: 0.0524.
It: 15500. Rec. loss: 0.0412.
It: 16000. Rec. loss: 0.0347

It: 16000. Rec. loss: 0.0405.
It: 16500. Rec. loss: 0.0379.
It: 17000. Rec. loss: 0.0376.
It: 17500. Rec. loss: 0.0334.
It: 18000. Rec. loss: 0.0336.
It: 18500. Rec. loss: 0.0381.
It: 19000. Rec. loss: 0.0369.
It: 19500. Rec. loss: 0.0349.
It: 20000. Rec. loss: 0.0377.
It: 20500. Rec. loss: 0.0374.
It: 21000. Rec. loss: 0.0362.
It: 21500. Rec. loss: 0.0296.
It: 22000. Rec. loss: 0.0273.
It: 22500. Rec. loss: 0.0263.
It: 23000. Rec. loss: 0.0264.
It: 23500. Rec. loss: 0.0246.
It: 23999. Rec. loss: 0.0156.
It: 0. Rec. loss: 0.1400.
It: 500. Rec. loss: 0.0619.
It: 1000. Rec. loss: 0.0601.
It: 1500. Rec. loss: 0.0555.
It: 2000. Rec. loss: 0.0579.
It: 2500. Rec. loss: 0.0614.
It: 3000. Rec. loss: 0.0613.
It: 3500. Rec. loss: 0.0596.
It: 4000. Rec. loss: 0.0643.
It: 4500. Rec. loss: 0.0565.
It: 5000. Rec. loss: 0.0629.
It: 5500. Rec. loss: 0.0588.
It: 6000. Rec. loss: 0.0642.
It: 6500. Rec. loss: 0.0622.
It: 7000. Rec. loss: 0.0582.
It: 7500. Rec. loss: 0.0634.
It: 8000. Rec. loss: 0.0609.
I

It: 4000. Rec. loss: 0.0383.
It: 4500. Rec. loss: 0.0391.
It: 5000. Rec. loss: 0.0371.
It: 5500. Rec. loss: 0.0400.
It: 6000. Rec. loss: 0.0393.
It: 6500. Rec. loss: 0.0402.
It: 7000. Rec. loss: 0.0440.
It: 7500. Rec. loss: 0.0438.
It: 8000. Rec. loss: 0.0396.
It: 8500. Rec. loss: 0.0389.
It: 9000. Rec. loss: 0.0430.
It: 9500. Rec. loss: 0.0323.
It: 10000. Rec. loss: 0.0326.
It: 10500. Rec. loss: 0.0322.
It: 11000. Rec. loss: 0.0318.
It: 11500. Rec. loss: 0.0339.
It: 12000. Rec. loss: 0.0330.
It: 12500. Rec. loss: 0.0371.
It: 13000. Rec. loss: 0.0369.
It: 13500. Rec. loss: 0.0317.
It: 14000. Rec. loss: 0.0322.
It: 14500. Rec. loss: 0.0323.
It: 15000. Rec. loss: 0.0321.
It: 15500. Rec. loss: 0.0260.
It: 16000. Rec. loss: 0.0224.
It: 16500. Rec. loss: 0.0209.
It: 17000. Rec. loss: 0.0223.
It: 17500. Rec. loss: 0.0188.
It: 18000. Rec. loss: 0.0193.
It: 18500. Rec. loss: 0.0209.
It: 19000. Rec. loss: 0.0202.
It: 19500. Rec. loss: 0.0191.
It: 20000. Rec. loss: 0.0197.
It: 20500. Rec. loss: 

...

It: 15000. Rec. loss: 0.0742.
It: 15500. Rec. loss: 0.0663.
It: 16000. Rec. loss: 0.0612.
It: 16500. Rec. loss: 0.0527.
It: 17000. Rec. loss: 0.0569.
It: 17500. Rec. loss: 0.0607.
It: 18000. Rec. loss: 0.0546.
It: 18500. Rec. loss: 0.0531.
It: 19000. Rec. loss: 0.0524.
It: 19500. Rec. loss: 0.0574.
It: 20000. Rec. loss: 0.0582.
It: 20500. Rec. loss: 0.0534.
It: 21000. Rec. loss: 0.0548.
It: 21500. Rec. loss: 0.0472.
It: 22000. Rec. loss: 0.0427.
It: 22500. Rec. loss: 0.0409.
It: 23000. Rec. loss: 0.0374.
It: 23500. Rec. loss: 0.0376.
It: 23999. Rec. loss: 0.0209.
It: 0. Rec. loss: 0.1786.
It: 500. Rec. loss: 0.0749.
It: 1000. Rec. loss: 0.0781.
It: 1500. Rec. loss: 0.0784.
It: 2000. Rec. loss: 0.0806.
It: 2500. Rec. loss: 0.0738.
It: 3000. Rec. loss: 0.0749.
It: 3500. Rec. loss: 0.0800.
It: 4000. Rec. loss: 0.0827.
It: 4500. Rec. loss: 0.0756.
It: 5000. Rec. loss: 0.0871.
It: 5500. Rec. loss: 0.0812.
It: 6000. Rec. loss: 0.0838.
It: 6500. Rec. loss: 0.0747.
It: 7000. Rec. loss: 0.0749.

It: 7000. Rec. loss: 0.0804.
It: 7500. Rec. loss: 0.0805.
It: 8000. Rec. loss: 0.0765.
It: 8500. Rec. loss: 0.0772.
It: 9000. Rec. loss: 0.0733.
It: 9500. Rec. loss: 0.0684.
It: 10000. Rec. loss: 0.0709.
It: 10500. Rec. loss: 0.0744.
It: 11000. Rec. loss: 0.0784.
It: 11500. Rec. loss: 0.0680.
It: 12000. Rec. loss: 0.0761.
It: 12500. Rec. loss: 0.0811.
It: 13000. Rec. loss: 0.0839.
It: 13500. Rec. loss: 0.0817.
It: 14000. Rec. loss: 0.0817.
It: 14500. Rec. loss: 0.0792.
It: 15000. Rec. loss: 0.0755.
It: 15500. Rec. loss: 0.0625.
It: 16000. Rec. loss: 0.0600.
It: 16500. Rec. loss: 0.0619.
It: 17000. Rec. loss: 0.0570.
It: 17500. Rec. loss: 0.0644.
It: 18000. Rec. loss: 0.0540.
It: 18500. Rec. loss: 0.0585.
It: 19000. Rec. loss: 0.0611.
It: 19500. Rec. loss: 0.0563.
It: 20000. Rec. loss: 0.0533.
It: 20500. Rec. loss: 0.0517.
It: 21000. Rec. loss: 0.0591.
It: 21500. Rec. loss: 0.0413.
It: 22000. Rec. loss: 0.0391.
It: 22500. Rec. loss: 0.0391.
It: 23000. Rec. loss: 0.0392.
It: 23500. Rec. 

It: 19000. Rec. loss: 0.0153.
It: 19500. Rec. loss: 0.0161.
It: 20000. Rec. loss: 0.0162.
It: 20500. Rec. loss: 0.0151.
It: 21000. Rec. loss: 0.0148.
It: 21500. Rec. loss: 0.0135.
It: 22000. Rec. loss: 0.0117.
It: 22500. Rec. loss: 0.0116.
It: 23000. Rec. loss: 0.0116.
It: 23500. Rec. loss: 0.0115.
It: 23999. Rec. loss: 0.0077.
It: 0. Rec. loss: 0.0727.
It: 500. Rec. loss: 0.0326.
It: 1000. Rec. loss: 0.0329.
It: 1500. Rec. loss: 0.0317.
It: 2000. Rec. loss: 0.0337.
It: 2500. Rec. loss: 0.0336.
It: 3000. Rec. loss: 0.0325.
It: 3500. Rec. loss: 0.0322.
It: 4000. Rec. loss: 0.0314.
It: 4500. Rec. loss: 0.0336.
It: 5000. Rec. loss: 0.0325.
It: 5500. Rec. loss: 0.0356.
It: 6000. Rec. loss: 0.0297.
It: 6500. Rec. loss: 0.0338.
It: 7000. Rec. loss: 0.0302.
It: 7500. Rec. loss: 0.0344.
It: 8000. Rec. loss: 0.0311.
It: 8500. Rec. loss: 0.0315.
It: 9000. Rec. loss: 0.0309.
It: 9500. Rec. loss: 0.0249.
It: 10000. Rec. loss: 0.0243.
It: 10500. Rec. loss: 0.0234.
It: 11000. Rec. loss: 0.0229.
It: 

It: 7000. Rec. loss: 0.0357.
It: 7500. Rec. loss: 0.0340.
It: 8000. Rec. loss: 0.0325.
It: 8500. Rec. loss: 0.0323.
It: 9000. Rec. loss: 0.0345.
It: 9500. Rec. loss: 0.0300.
It: 10000. Rec. loss: 0.0263.
It: 10500. Rec. loss: 0.0299.
It: 11000. Rec. loss: 0.0241.
It: 11500. Rec. loss: 0.0270.
It: 12000. Rec. loss: 0.0276.
It: 12500. Rec. loss: 0.0255.
It: 13000. Rec. loss: 0.0251.
It: 13500. Rec. loss: 0.0255.
It: 14000. Rec. loss: 0.0264.
It: 14500. Rec. loss: 0.0265.
It: 15000. Rec. loss: 0.0265.
It: 15500. Rec. loss: 0.0223.
It: 16000. Rec. loss: 0.0190.
It: 16500. Rec. loss: 0.0181.
It: 17000. Rec. loss: 0.0179.
It: 17500. Rec. loss: 0.0179.
It: 18000. Rec. loss: 0.0180.
It: 18500. Rec. loss: 0.0177.
It: 19000. Rec. loss: 0.0173.
It: 19500. Rec. loss: 0.0175.
It: 20000. Rec. loss: 0.0175.
It: 20500. Rec. loss: 0.0171.
It: 21000. Rec. loss: 0.0176.
It: 21500. Rec. loss: 0.0151.
It: 22000. Rec. loss: 0.0133.
It: 22500. Rec. loss: 0.0128.
It: 23000. Rec. loss: 0.0121.
It: 23500. Rec. 

It: 23500. Rec. loss: 0.0119.
It: 23999. Rec. loss: 0.0087.
It: 0. Rec. loss: 0.0724.
It: 500. Rec. loss: 0.0355.
It: 1000. Rec. loss: 0.0324.
It: 1500. Rec. loss: 0.0327.
It: 2000. Rec. loss: 0.0328.
It: 2500. Rec. loss: 0.0331.
It: 3000. Rec. loss: 0.0329.
It: 3500. Rec. loss: 0.0331.
It: 4000. Rec. loss: 0.0348.
It: 4500. Rec. loss: 0.0355.
It: 5000. Rec. loss: 0.0328.
It: 5500. Rec. loss: 0.0336.
It: 6000. Rec. loss: 0.0325.
It: 6500. Rec. loss: 0.0348.
It: 7000. Rec. loss: 0.0325.
It: 7500. Rec. loss: 0.0329.
It: 8000. Rec. loss: 0.0336.
It: 8500. Rec. loss: 0.0326.
It: 9000. Rec. loss: 0.0358.
It: 9500. Rec. loss: 0.0292.
It: 10000. Rec. loss: 0.0256.
It: 10500. Rec. loss: 0.0269.
It: 11000. Rec. loss: 0.0243.
It: 11500. Rec. loss: 0.0276.
It: 12000. Rec. loss: 0.0294.
It: 12500. Rec. loss: 0.0294.
It: 13000. Rec. loss: 0.0265.
It: 13500. Rec. loss: 0.0269.
It: 14000. Rec. loss: 0.0262.
It: 14500. Rec. loss: 0.0257.
It: 15000. Rec. loss: 0.0244.
It: 15500. Rec. loss: 0.0193.
It: 

In [10]:
print("Total no. of Images: ",count)
print("Attack Success Rate: ", ASR/count)
print("Avg. MSE: ",test_mse_final.item()/count)
print("Avg. SSIM: ",sSim_final/count)
print("Avg. loss value: ",loss_value/count)
print("Avg. Duration to recover: ",whole_duration/count)


Total no. of Images:  100
Attack Success Rate:  0.73
Avg. MSE:  0.8811457824707031
Avg. SSIM:  0.9214599233865738
Avg. loss value:  0.010387026071548462
Avg. Duration to recover:  5953.77839974165


In [ ]:
test_mse_final=0
sSim_final=0
whole_duration=0
loss_value=0
count=0
ASR=0
for i in range (0, len(trainloader.dataset),190):
    print("Image No.", count+1)
    print("For image : ", i)
    
    idx = i 
    # 8112 # the beagle
    # 1200 # the owl
    # 11794 # the German shepherd
    # 19449 # the panda


    # np.random.randint(len(validloader.dataset))

    img, label = trainloader.dataset[idx]
    labels = torch.as_tensor((label,), device=setup['device'])
    ground_truth = img.to(**setup).unsqueeze(0)
   
    #plot(ground_truth);
    print([trainloader.dataset.classes[l] for l in labels]);


    ground_truth_denormalized = torch.clamp(ground_truth * ds + dm, 0, 1)
    output_path = os.path.join(output_folder, f'{idx}_{arch}_Brain_MRI_untrained_input_loop_test.png')
    torchvision.utils.save_image(ground_truth_denormalized, output_path)
    
    
    
    #torchvision.utils.save_image(ground_truth_denormalized, f'{idx}_{arch}_covid_19_untrained_input_loop_test.png')

    
    model.zero_grad()
    target_loss, _, _ = loss_fn(model(ground_truth), labels)
    input_gradient = torch.autograd.grad(target_loss, model.parameters())
    input_gradient = [grad.detach() for grad in input_gradient]
    full_norm = torch.stack([g.norm() for g in input_gradient]).mean()
    print(f'Full gradient norm is {full_norm:e}.')
    config = dict(signed=True,
                  boxed=True,
                  cost_fn='sim',
                  indices='top10',
                  weights='equal',
                  lr=0.1,
                  optim='adam',
                  restarts=8,
                  max_iterations=24000,
                  total_variation=1e-6,
                  init='randn',
                  filter='median',
                  lr_decay=True,
                  scoring_choice='loss')
    start_time = time.time()
    rec_machine = inversefed.GradientReconstructor(model, (dm, ds), config, num_images=1)
    output, stats = rec_machine.reconstruct(input_gradient, labels, img_shape=(3, 224, 224))
    
    test_mse = (output.detach() - ground_truth).pow(2).mean()
    test_mse_final=test_mse_final+test_mse
    feat_mse = (model(output.detach())- model(ground_truth)).pow(2).mean()  
    test_psnr = inversefed.metrics.psnr(output, ground_truth, factor=1/ds)
    sSim=ssim(output, ground_truth).item()
    sSim_final=sSim_final+sSim
    loss_value=loss_value+stats['opt']
    plot(output)
    
    plt.title(f"Rec. loss: {stats['opt']:2.4f} | MSE: {test_mse:2.4f} | SSIM: {sSim:2.4f}"
              f"| PSNR: {test_psnr:4.2f} | FMSE: {feat_mse:2.4e} |");
    
    output_denormalized = torch.clamp(output * ds + dm, 0, 1)
    output_path = os.path.join(output_folder, f'{idx}_{arch}_Brain_MRI_untrained_output_loop_test.png')
    torchvision.utils.save_image(output_denormalized, output_path)
    #torchvision.utils.save_image(output_denormalized, f'{idx}_{arch}_covid_19_untrained_output_loop_test.png')
    
    
    

    
    data = inversefed.metrics.activation_errors(model, output, ground_truth)
    fig, axes = plt.subplots(2, 4, sharey=False, figsize=(14,8))
    axes[0, 0].semilogy(list(data['se'].values())[:-3])
    axes[0, 0].set_title('SE')
    axes[0, 1].semilogy(list(data['mse'].values())[:-3])
    axes[0, 1].set_title('MSE')
    axes[0, 2].plot(list(data['sim'].values())[:-3])
    axes[0, 2].set_title('Similarity')

    convs = [val for key, val in data['mse'].items() if 'conv' in key]
    axes[1, 0].semilogy(convs)
    axes[1, 0].set_title('MSE - conv layers')
    convs = [val for key, val in data['mse'].items() if 'conv1' in key]
    axes[1, 1].semilogy(convs)
    convs = [val for key, val in data['mse'].items() if 'conv2' in key]
    axes[1, 1].semilogy(convs)
    axes[1, 1].set_title('MSE - conv1 vs conv2 layers')
    bns = [val for key, val in data['mse'].items() if 'bn' in key]
    axes[1, 2].plot(bns)
    axes[1, 2].set_title('MSE - bn layers')
    fig.suptitle('Error between layers');
    
    
    end_time=time.time()-.1
    duration=end_time-start_time
    whole_duration=whole_duration+duration
    
    print("Time to recover Image", i, ": ",duration)
    if(sSim>=0.9):
        ASR=ASR+1
    print()
    count=count+1
    
    

Image No. 1
For image :  0
[('tench', 'Tinca tinca')]


/lclhome/bdas004/anaconda3/envs/GPU_env/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Full gradient norm is 3.140548e+00.
It: 0. Rec. loss: 0.0676.
It: 500. Rec. loss: 0.0415.
It: 1000. Rec. loss: 0.0442.
It: 1500. Rec. loss: 0.0416.
It: 2000. Rec. loss: 0.0428.
It: 2500. Rec. loss: 0.0401.
It: 3000. Rec. loss: 0.0426.
It: 3500. Rec. loss: 0.0432.
It: 4000. Rec. loss: 0.0421.
It: 4500. Rec. loss: 0.0421.
It: 5000. Rec. loss: 0.0402.
It: 5500. Rec. loss: 0.0395.
It: 6000. Rec. loss: 0.0420.
It: 6500. Rec. loss: 0.0429.
It: 7000. Rec. loss: 0.0449.
It: 7500. Rec. loss: 0.0415.
It: 8000. Rec. loss: 0.0421.
It: 8500. Rec. loss: 0.0451.
It: 9000. Rec. loss: 0.0418.
It: 9500. Rec. loss: 0.0370.
It: 10000. Rec. loss: 0.0338.
It: 10500. Rec. loss: 0.0349.
It: 11000. Rec. loss: 0.0346.
It: 11500. Rec. loss: 0.0311.
It: 12000. Rec. loss: 0.0333.
It: 12500. Rec. loss: 0.0327.
It: 13000. Rec. loss: 0.0332.
It: 13500. Rec. loss: 0.0345.
It: 14000. Rec. loss: 0.0318.
It: 14500. Rec. loss: 0.0324.
It: 15000. Rec. loss: 0.0316.
It: 15500. Rec. loss: 0.0248.
It: 16000. Rec. loss: 0.0232

It: 16000. Rec. loss: 0.0251.
It: 16500. Rec. loss: 0.0229.
It: 17000. Rec. loss: 0.0240.
It: 17500. Rec. loss: 0.0216.
It: 18000. Rec. loss: 0.0223.
It: 18500. Rec. loss: 0.0209.
It: 19000. Rec. loss: 0.0212.
It: 19500. Rec. loss: 0.0232.
It: 20000. Rec. loss: 0.0220.
It: 20500. Rec. loss: 0.0221.
It: 21000. Rec. loss: 0.0222.
It: 21500. Rec. loss: 0.0180.
It: 22000. Rec. loss: 0.0165.
It: 22500. Rec. loss: 0.0158.
It: 23000. Rec. loss: 0.0153.
It: 23500. Rec. loss: 0.0145.
It: 23999. Rec. loss: 0.0100.
It: 0. Rec. loss: 0.0692.
It: 500. Rec. loss: 0.0425.
It: 1000. Rec. loss: 0.0422.
It: 1500. Rec. loss: 0.0439.
It: 2000. Rec. loss: 0.0429.
It: 2500. Rec. loss: 0.0393.
It: 3000. Rec. loss: 0.0385.
It: 3500. Rec. loss: 0.0406.
It: 4000. Rec. loss: 0.0414.
It: 4500. Rec. loss: 0.0389.
It: 5000. Rec. loss: 0.0402.
It: 5500. Rec. loss: 0.0390.
It: 6000. Rec. loss: 0.0410.
It: 6500. Rec. loss: 0.0409.
It: 7000. Rec. loss: 0.0431.
It: 7500. Rec. loss: 0.0398.
It: 8000. Rec. loss: 0.0433.
I

It: 4000. Rec. loss: 0.0361.
It: 4500. Rec. loss: 0.0353.
It: 5000. Rec. loss: 0.0334.
It: 5500. Rec. loss: 0.0376.
It: 6000. Rec. loss: 0.0358.
It: 6500. Rec. loss: 0.0334.
It: 7000. Rec. loss: 0.0374.
It: 7500. Rec. loss: 0.0335.
It: 8000. Rec. loss: 0.0361.
It: 8500. Rec. loss: 0.0372.
It: 9000. Rec. loss: 0.0335.
It: 9500. Rec. loss: 0.0295.
It: 10000. Rec. loss: 0.0299.
It: 10500. Rec. loss: 0.0281.
It: 11000. Rec. loss: 0.0281.
It: 11500. Rec. loss: 0.0293.
It: 12000. Rec. loss: 0.0283.
It: 12500. Rec. loss: 0.0304.
It: 13000. Rec. loss: 0.0262.
It: 13500. Rec. loss: 0.0285.
It: 14000. Rec. loss: 0.0277.
It: 14500. Rec. loss: 0.0272.
It: 15000. Rec. loss: 0.0268.
It: 15500. Rec. loss: 0.0197.
It: 16000. Rec. loss: 0.0185.
It: 16500. Rec. loss: 0.0192.
It: 17000. Rec. loss: 0.0184.
It: 17500. Rec. loss: 0.0169.
It: 18000. Rec. loss: 0.0170.
It: 18500. Rec. loss: 0.0177.
It: 19000. Rec. loss: 0.0164.
It: 19500. Rec. loss: 0.0174.
It: 20000. Rec. loss: 0.0174.
It: 20500. Rec. loss: 

...

It: 9000. Rec. loss: 0.0401.
It: 9500. Rec. loss: 0.0374.
It: 10000. Rec. loss: 0.0390.
It: 10500. Rec. loss: 0.0364.
It: 11000. Rec. loss: 0.0367.
It: 11500. Rec. loss: 0.0379.
It: 12000. Rec. loss: 0.0360.
It: 12500. Rec. loss: 0.0361.
It: 13000. Rec. loss: 0.0390.
It: 13500. Rec. loss: 0.0385.
It: 14000. Rec. loss: 0.0369.
It: 14500. Rec. loss: 0.0340.
It: 15000. Rec. loss: 0.0345.
It: 15500. Rec. loss: 0.0245.
It: 16000. Rec. loss: 0.0252.
It: 16500. Rec. loss: 0.0239.
It: 17000. Rec. loss: 0.0247.
It: 17500. Rec. loss: 0.0231.
It: 18000. Rec. loss: 0.0223.
It: 18500. Rec. loss: 0.0223.
It: 19000. Rec. loss: 0.0213.
It: 19500. Rec. loss: 0.0196.
It: 20000. Rec. loss: 0.0201.
It: 20500. Rec. loss: 0.0211.
It: 21000. Rec. loss: 0.0211.
It: 21500. Rec. loss: 0.0166.
It: 22000. Rec. loss: 0.0150.
It: 22500. Rec. loss: 0.0135.
It: 23000. Rec. loss: 0.0128.
It: 23500. Rec. loss: 0.0129.
It: 23999. Rec. loss: 0.0090.
It: 0. Rec. loss: 0.0886.
It: 500. Rec. loss: 0.0427.
It: 1000. Rec. los

It: 1000. Rec. loss: 0.0431.
It: 1500. Rec. loss: 0.0420.
It: 2000. Rec. loss: 0.0420.
It: 2500. Rec. loss: 0.0449.
It: 3000. Rec. loss: 0.0428.
It: 3500. Rec. loss: 0.0415.
It: 4000. Rec. loss: 0.0422.
It: 4500. Rec. loss: 0.0416.
It: 5000. Rec. loss: 0.0418.
It: 5500. Rec. loss: 0.0440.
It: 6000. Rec. loss: 0.0430.
It: 6500. Rec. loss: 0.0393.
It: 7000. Rec. loss: 0.0419.
It: 7500. Rec. loss: 0.0404.
It: 8000. Rec. loss: 0.0413.
It: 8500. Rec. loss: 0.0392.
It: 9000. Rec. loss: 0.0432.
It: 9500. Rec. loss: 0.0369.
It: 10000. Rec. loss: 0.0328.
It: 10500. Rec. loss: 0.0339.
It: 11000. Rec. loss: 0.0359.
It: 11500. Rec. loss: 0.0350.
It: 12000. Rec. loss: 0.0353.
It: 12500. Rec. loss: 0.0337.
It: 13000. Rec. loss: 0.0366.
It: 13500. Rec. loss: 0.0359.
It: 14000. Rec. loss: 0.0362.
It: 14500. Rec. loss: 0.0365.
It: 15000. Rec. loss: 0.0369.
It: 15500. Rec. loss: 0.0247.
It: 16000. Rec. loss: 0.0219.
It: 16500. Rec. loss: 0.0234.
It: 17000. Rec. loss: 0.0215.
It: 17500. Rec. loss: 0.0235

It: 13500. Rec. loss: 0.0284.
It: 14000. Rec. loss: 0.0274.
It: 14500. Rec. loss: 0.0297.
It: 15000. Rec. loss: 0.0291.
It: 15500. Rec. loss: 0.0225.
It: 16000. Rec. loss: 0.0209.
It: 16500. Rec. loss: 0.0191.
It: 17000. Rec. loss: 0.0197.
It: 17500. Rec. loss: 0.0195.
It: 18000. Rec. loss: 0.0206.
It: 18500. Rec. loss: 0.0187.
It: 19000. Rec. loss: 0.0174.
It: 19500. Rec. loss: 0.0187.
It: 20000. Rec. loss: 0.0187.
It: 20500. Rec. loss: 0.0176.
It: 21000. Rec. loss: 0.0165.
It: 21500. Rec. loss: 0.0144.
It: 22000. Rec. loss: 0.0130.
It: 22500. Rec. loss: 0.0125.
It: 23000. Rec. loss: 0.0122.
It: 23500. Rec. loss: 0.0121.
It: 23999. Rec. loss: 0.0087.
It: 0. Rec. loss: 0.0703.
It: 500. Rec. loss: 0.0329.
It: 1000. Rec. loss: 0.0333.
It: 1500. Rec. loss: 0.0341.
It: 2000. Rec. loss: 0.0336.
It: 2500. Rec. loss: 0.0316.
It: 3000. Rec. loss: 0.0343.
It: 3500. Rec. loss: 0.0337.
It: 4000. Rec. loss: 0.0315.
It: 4500. Rec. loss: 0.0324.
It: 5000. Rec. loss: 0.0328.
It: 5500. Rec. loss: 0.03

It: 1500. Rec. loss: 0.0387.
It: 2000. Rec. loss: 0.0437.
It: 2500. Rec. loss: 0.0426.
It: 3000. Rec. loss: 0.0405.
It: 3500. Rec. loss: 0.0417.
It: 4000. Rec. loss: 0.0431.
It: 4500. Rec. loss: 0.0390.
It: 5000. Rec. loss: 0.0456.
It: 5500. Rec. loss: 0.0407.
It: 6000. Rec. loss: 0.0421.
It: 6500. Rec. loss: 0.0421.
It: 7000. Rec. loss: 0.0401.
It: 7500. Rec. loss: 0.0393.
It: 8000. Rec. loss: 0.0400.
It: 8500. Rec. loss: 0.0427.
It: 9000. Rec. loss: 0.0401.
It: 9500. Rec. loss: 0.0336.
It: 10000. Rec. loss: 0.0325.
It: 10500. Rec. loss: 0.0331.
It: 11000. Rec. loss: 0.0323.
It: 11500. Rec. loss: 0.0310.
It: 12000. Rec. loss: 0.0340.
It: 12500. Rec. loss: 0.0345.
It: 13000. Rec. loss: 0.0329.
It: 13500. Rec. loss: 0.0319.
It: 14000. Rec. loss: 0.0321.
It: 14500. Rec. loss: 0.0304.
It: 15000. Rec. loss: 0.0309.
It: 15500. Rec. loss: 0.0240.
It: 16000. Rec. loss: 0.0224.
It: 16500. Rec. loss: 0.0200.
It: 17000. Rec. loss: 0.0201.
It: 17500. Rec. loss: 0.0216.
It: 18000. Rec. loss: 0.020

It: 18000. Rec. loss: 0.0208.
It: 18500. Rec. loss: 0.0194.
It: 19000. Rec. loss: 0.0212.
It: 19500. Rec. loss: 0.0198.
It: 20000. Rec. loss: 0.0213.
It: 20500. Rec. loss: 0.0240.
It: 21000. Rec. loss: 0.0218.
It: 21500. Rec. loss: 0.0167.
It: 22000. Rec. loss: 0.0152.
It: 22500. Rec. loss: 0.0155.
It: 23000. Rec. loss: 0.0145.
It: 23500. Rec. loss: 0.0132.
It: 23999. Rec. loss: 0.0092.
Choosing optimal result ...
Optimal result score: 0.0123
Total time: 31943.595656633377.
Average Time:
Time to recover Image 1140 :  31943.96714735031

Image No. 8
For image :  1330
[('goldfish', 'Carassius auratus')]
Full gradient norm is 2.894802e+00.
It: 0. Rec. loss: 0.0786.
It: 500. Rec. loss: 0.0321.
It: 1000. Rec. loss: 0.0299.
It: 1500. Rec. loss: 0.0351.
It: 2000. Rec. loss: 0.0339.
It: 2500. Rec. loss: 0.0319.
It: 3000. Rec. loss: 0.0298.
It: 3500. Rec. loss: 0.0313.
It: 4000. Rec. loss: 0.0312.
It: 4500. Rec. loss: 0.0305.
It: 5000. Rec. loss: 0.0296.
It: 5500. Rec. loss: 0.0293.
It: 6000. Re

In [1]:
print("Total no. of Images: ",count)
print("Attack Success Rate: ", ASR/count)
print("Avg. MSE: ",test_mse_final.item()/count)
print("Avg. SSIM: ",sSim_final/count)
print("Avg. loss value: ",loss_value/count)
print("Avg. Duration to recover: ",whole_duration/count)


NameError: name 'count' is not defined